In [9]:
import sofar as sf
import pyfar as pf
from pathlib import Path

input_sofa = Path("../assets") / "pp2_HRIRs_measured.sofa"

# read Sofa file

In [6]:
sofa = sf.read_sofa(input_sofa)

In [16]:
sofa.inspect()

SimpleFreeFieldHRIR 1.0 (SOFA version 1.0)
-------------------------------------------
GLOBAL_Conventions : SOFA
GLOBAL_Version : 1.0
GLOBAL_SOFAConventions : SimpleFreeFieldHRIR
GLOBAL_SOFAConventionsVersion : 1.0
GLOBAL_APIName : ARI SOFA API for Matlab/Octave
GLOBAL_APIVersion : 1.0.3
GLOBAL_AuthorContact : fabian.brinkmann@mailbox.org; manoj.fma@gmail.com
GLOBAL_DataType : FIR
GLOBAL_License : cc-by 4.0 (https://creativecommons.org/licenses/by/4.0/)
GLOBAL_Organization : Audio Communication Group, Technical University Berlin; HUAWEI Munich Research Center; Sennheiser
GLOBAL_RoomType : free field
GLOBAL_DateCreated : 2018-09-17 15:28:15
GLOBAL_DateModified : 2018-10-23 16:06:34
GLOBAL_Title : head-related impulse responses
GLOBAL_DatabaseName : HUTUBS
GLOBAL_ListenerShortName : pp2
ListenerPosition : (I=1, C=3)
  [0. 0. 0.]
ListenerPosition_Type : cartesian
ListenerPosition_Units : metre
ReceiverPosition : (R=2, C=3, I=1)
  [[ 0.    0.75  0.  ]
   [ 0.   -0.75  0.  ]]
ReceiverPositi

In [32]:
start_times = []
for i, source in enumerate(sofa.Data_IR):
    for j, ir in enumerate(source):
        ir_signal = pf.Signal(ir, sampling_rate=sofa.Data_SamplingRate)
        start_times.append(pf.dsp.find_impulse_response_start(ir_signal))
        
print(min(start_times), max(start_times))

shift_to = min(start_times)

7 47


In [33]:
delay_times = []
time_shifted_irs = []
for i, source in enumerate(sofa.Data_IR):
    source_irs = []
    source_delays = []
    for j, ir in enumerate(source):
        ir_signal = pf.Signal(ir, sampling_rate=sofa.Data_SamplingRate)
        ir_start = pf.dsp.find_impulse_response_start(ir_signal)
        ir_delay = ir_start - shift_to
        pf.dsp.time_shift(ir_signal, -ir_delay)
        source_irs.append(ir_signal.time)
        source_delays.append(ir_delay)

    delay_times.append(source_delays)
    time_shifted_irs.append(source_irs)
        


In [34]:
sofa.Data_Delay = delay_times
sofa.Data_IR = time_shifted_irs

In [36]:
sofa.verify()